In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time

from threading import Thread
from threading import Lock

import multiprocessing

from sklearn.model_selection import train_test_split 

from src.Images import preprocess_images_from_dir
from src.Images import get_all_images_path
from src.Images import read_processed_image
from src.Images import plot_image
from src.Images import flatten_to_image

from src.Distances import euclidean_distance
from src.Distances import cosine_distance

from src.Knn import Knn

from src.Settings import *

from src.Parallel import thread_classify_function
from src.Parallel import split_data_into_chunks
from src.Parallel import presentation_thread_classify_function

print_all_settings()

USE_NUMPY_ARRAY: True
N_ROWS: 32
N_COLUMNS: 32
PREPROCESS_IMAGES: False
N_THREADS: 4
N_TESTS_USED: 0
WORKSPACE: .
DATASET_DIR: ./dataset
IMAGES_DIR: ./dataset/dataset
PROCESSED_IMAGES_PATH: ./dataset/dataset/_processed


In [2]:
images_path = get_all_images_path(GlobalSettings.PROCESSED_IMAGES_PATH)
images_path[:3]

['./dataset/dataset/_processed/14719 flat tile corner 2x2 000L.png',
 './dataset/dataset/_processed/14719 flat tile corner 2x2 000R.png',
 './dataset/dataset/_processed/14719 flat tile corner 2x2 001L.png']

In [3]:
data = [
    (
        int(path.split(" ")[0].split("/")[-1]),
        path,
    ) for path in images_path
]

In [4]:
y, x = zip(*data)
x = list(x)
y = list(y)

In [6]:
x_train_paths, x_test_paths, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

In [7]:
x_train_original = [np.reshape(read_processed_image(path, GlobalSettings.N_ROWS, GlobalSettings.N_COLUMNS).flatten(),(-1,1)) for path in x_train_paths]

In [8]:
len(x_train_original)

28000

In [9]:
x_train_original[0].shape

(1024, 1)

In [10]:
M = x_train_original[0]
for i in range(1, len(x_train_original)):
    M = np.hstack((M, x_train_original[i]))

print(M.shape)
M

(1024, 28000)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
M_mean = np.mean(M, axis=0)
print(M_mean.shape)
M_mean

(28000,)


array([0.02754672, 0.01830959, 0.02506893, ..., 0.03283931, 0.04458487,
       0.1147595 ])

In [12]:
M = M - M_mean

In [13]:
u, s, vt = np.linalg.svd(M,full_matrices=False)

print("Dimensões de u:", u.shape)
print("Dimensões de s:", s.shape)
print("Dimensões de vt:", vt.shape)

Dimensões de u: (1024, 1024)
Dimensões de s: (1024,)
Dimensões de vt: (1024, 28000)


In [14]:
r = 100 
lista_imagens_projetadas = []

for i in range(len(x_train_paths)):
    path = x_train_paths[i]
    imag = np.reshape(read_processed_image(path, GlobalSettings.N_ROWS, GlobalSettings.N_COLUMNS).flatten(),(-1,1))
    imag_proj = u[:,0:r].T@imag
    lista_imagens_projetadas.append(imag_proj)

In [15]:
len(lista_imagens_projetadas)

28000

In [16]:
acertos = 0
erros = 0

for path_index in range(len(x_test_paths)):
    path = x_test_paths[path_index]
    imagemTeste = np.reshape(read_processed_image(path, GlobalSettings.N_ROWS, GlobalSettings.N_COLUMNS).flatten(),(-1,1))
    imagemTesteProjetada = u[:,0:r].T@imagemTeste
    distancia = 5000000
    index=-1
    for i in range(len(lista_imagens_projetadas)): 
        dist = np.linalg.norm(imagemTesteProjetada-lista_imagens_projetadas[i])
        if dist < distancia:
            distancia = dist
            index = i

    if y_test[path_index] == y_train[index]:
        acertos += 1
    else:
        erros += 1
        
total = acertos+erros

In [17]:
print("Quantidade de acertos:", acertos)
print("Quantidade de erros:", erros)
print("Total:", total)
print("Acurácia: {:.2f}%".format(100*(acertos/total)))

Quantidade de acertos: 7284
Quantidade de erros: 4716
Total: 12000
Acurácia: 60.70%
